In [1]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y= tf.Variable(4, name="y")

In [2]:
f=x*x*y +y +2

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [4]:
sess.close()

In [5]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    results=f.eval()
print(results)

42


In [6]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()

In [7]:
init.run()
result3 = f.eval()
print(result3)

42


In [8]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [9]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [10]:
x2.graph is graph

True

In [11]:
x2.graph is tf.get_default_graph()

False

In [12]:
sess.close()

In [13]:
import numpy as np 
from sklearn.datasets import fetch_california_housing

In [14]:
housing = fetch_california_housing()

m, n = housing.data.shape

In [15]:
m, n = housing.data.shape

In [16]:
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [17]:
#linear regression
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [18]:
with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


In [19]:
#scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [20]:
#batch gradient descent
n_epochs = 1000
learning_rate = 0.01


#full data training
#X= tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
#y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")

#mini batch

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name= "theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
#manual gradient
#gradients = 2/m * tf.matmul(tf.transpose(X), error) 
#using tf autodiff for gradients
#gradients = tf.gradients(mse, [theta])[0]
#training_op = tf.assign(theta, theta - learning_rate * gradients)

#using built in optimizers instead of manual op definition
optimizer= tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)


In [21]:

init = tf.global_variables_initializer()
# save model state
saver = tf.train.Saver()

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size = batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

with tf.Session() as sess: 
    sess.run(init)
    
    for epoch in range(n_epochs):
        #without batch
        #if epoch % 100 == 0:
           # print("epoch", epoch, "MSE=", mse.eval())
         #sess.run(training_op)
        
        #with batch
        if epoch % 100 == 0:
            save_path= saver.save(sess, "/tmp/my_model.ckpt")
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()
    print(best_theta)   
    save_path= saver.save(sess, "/tmp/my_model_final.ckpt")

[[ 2.0714476 ]
 [ 0.8462012 ]
 [ 0.11558535]
 [-0.26835832]
 [ 0.32982782]
 [ 0.00608358]
 [ 0.07052915]
 [-0.87988573]
 [-0.8634251 ]]


In [22]:
#using placeholders to store data for feeding
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A: [[4,5,6], [7,8,9]]})

In [23]:
print(B_val_1)

[[6. 7. 8.]]


In [24]:
print(B_val_2)


[[ 9. 10. 11.]
 [12. 13. 14.]]


In [25]:

#restore model + vars
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [26]:
np.allclose(best_theta, best_theta_restored)

True

In [27]:


saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")  # this loads the graph structure
theta = tf.get_default_graph().get_tensor_by_name("theta:0") 

with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")  # this restores the graph's state
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [28]:
np.allclose(best_theta, best_theta_restored)

True

In [29]:
reset_graph()

In [30]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir= "tf_logs"
logdir="{}/run-{}/".format(root_logdir, now)

In [31]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name = "X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1],-1.0,1.0, seed=42), name = "theta")
y_pred = tf.matmul(X, theta, name="predictions")
with tf.name_scope("loss") as scope:
    error = y_pred-y
    mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()


In [32]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer= tf.summary.FileWriter(logdir, tf.get_default_graph())


In [33]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [34]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range (n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    
    best_theta= theta.eval()

In [35]:
file_writer.close()

In [36]:
best_theta

array([[ 2.0714476 ],
       [ 0.8462012 ],
       [ 0.11558535],
       [-0.26835832],
       [ 0.32982782],
       [ 0.00608358],
       [ 0.07052915],
       [-0.87988573],
       [-0.8634251 ]], dtype=float32)

In [37]:
print(error.op.name)

loss/sub


In [45]:
reset_graph()


In [46]:
def relu(X):
    with tf.name_scope("relu"):
        w_shape=(int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name = "bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, 0., name="max")

In [47]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

In [48]:
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")